In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
log_csv = pd.read_csv('log2.csv')
log_df = pd.DataFrame(log_csv)
log_df

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
0,57222,53,54587,53,allow,177,94,83,2,30,1,1
1,56258,3389,56258,3389,allow,4768,1600,3168,19,17,10,9
2,6881,50321,43265,50321,allow,238,118,120,2,1199,1,1
3,50553,3389,50553,3389,allow,3327,1438,1889,15,17,8,7
4,50002,443,45848,443,allow,25358,6778,18580,31,16,13,18
...,...,...,...,...,...,...,...,...,...,...,...,...
65527,63691,80,13237,80,allow,314,192,122,6,15,4,2
65528,50964,80,13485,80,allow,4680740,67312,4613428,4675,77,985,3690
65529,54871,445,0,0,drop,70,70,0,1,0,1,0
65530,54870,445,0,0,drop,70,70,0,1,0,1,0


### Определить какая служба(порт) заблокирована у получателя
Сессии, имеющие action=drop

In [36]:
drop_actions_df = log_df[log_df['Action'] == 'drop']
drop_actions_df

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
141,51048,445,0,0,drop,70,70,0,1,0,1,0
142,51045,445,0,0,drop,70,70,0,1,0,1,0
145,55725,445,0,0,drop,70,70,0,1,0,1,0
146,55723,445,0,0,drop,70,70,0,1,0,1,0
147,55724,445,0,0,drop,70,70,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
65519,49784,445,0,0,drop,70,70,0,1,0,1,0
65520,49783,445,0,0,drop,70,70,0,1,0,1,0
65529,54871,445,0,0,drop,70,70,0,1,0,1,0
65530,54870,445,0,0,drop,70,70,0,1,0,1,0


In [40]:
count_all_dst_ports = len(log_df['Destination Port'].unique())
count_drop_dst_ports = len(drop_actions_df['Destination Port'].unique())
print('Всего уникальных dst_port:', count_all_dst_ports, '\nДропнутые_dst_port:', count_drop_dst_ports, drop_actions_df['Destination Port'].unique(), 'SMB')

Всего уникальных dst_port: 3273 
Дропнутые_dst_port: 1 [445] SMB


### Соотношение количества заблокированных обращений к общему количеству

In [46]:
count_of_sessions = len(log_df)
count_of_sessions_for_smb_port = len(log_df[log_df['Action'] == 'drop'])
round(count_of_sessions_for_smb_port/count_of_sessions*100, 2), '%'

(19.61, '%')

### Количество портов отправителей. Разбиение(визуализация) на диапазоны
Уникальные src_port

In [47]:
unique_src_ports = pd.DataFrame(log_df['Source Port'].unique())
unique_src_ports.columns = ['unique_port']
# Кластеризация по диапазону
interval_port = []
for entry in unique_src_ports['unique_port']:
    if entry < 1023:
        interval_port.append('standart')
    elif entry > 1023 and entry < 49151:
        interval_port.append('registered')
    elif entry > 49151:
        interval_port.append('dynamic')
unique_src_ports.insert(loc= 1 , column='interval_status', value=interval_port) # Добавление колонки с названием диапазона
unique_src_ports.groupby(unique_src_ports['interval_status']).count() # Количество уникальных портоов по каждому диапазону

,unique_port
interval_status,
dynamic,14405
registered,8301
standart,18


### Оценить аномалии при обращениях. Учитывать время передачи и количество отправленного(количество пакетов или вес данных)

In [35]:
anomalies_in_traffic_df = log_df[log_df['Action'] == 'allow'][['Source Port', 'Destination Port', 'Bytes', 'Elapsed Time (sec)']]
anomalies_in_traffic_df

,Source Port,Destination Port,Bytes,Elapsed Time (sec)
0,57222,53,177,30
1,56258,3389,4768,17
2,6881,50321,238,1199
3,50553,3389,3327,17
4,50002,443,25358,16
...,...,...,...,...
65512,56985,53,711,31
65525,65323,53,356,30
65526,51710,43069,70,8
65527,63691,80,314,15
